## 1. Write a python program which searches all the product under a particular product from www.amazon.in.  The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search  for guitars.

In [1]:
#import necessary libraries
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException   #importing the exception
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import re
import time


import warnings
warnings.filterwarnings('ignore')

In [3]:
#connect to chrome browser
driver = webdriver.Chrome(r"D:\CHROME DOWNLODE\chromedriver\chromedriver.exe")
driver.get('http://www.amazon.in/')
delay = 10


In [4]:
#Search input
search_field_input = driver.find_element(By.XPATH,'//*[@id="twotabsearchtextbox"]')

search_field_input.send_keys(input('Enter the product name: '))

Enter the product name: guitar


In [5]:
#Clicking the search button
search_button = driver.find_element(By.ID,'nav-search-submit-button')
search_button.click()

## 2. In the above question, now scrape the following details of each product listed in first 3 pages of your  search results and save it in a data frame and csv. In case if any product has less than 3 pages in search  results then scrape all the products available under that product name. Details to be scraped are: "Brand  Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and  “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [6]:
#Clicking the next button to get the page url for iteration
next_button = driver.find_element(By.XPATH,"//a[@class = 's-pagination-item s-pagination-next s-pagination-button s-pagination-separator']")
next_button.click()

In [7]:
#Getting the url for iteration.
curren_url = driver.current_url
keyword = re.compile(r'https://www\.amazon\.in/s\?k=[a-zA-Z0-9]+&page=')
aurl = keyword.finditer(curren_url)
for i in aurl:
    a = i.group(0)
url = []
for page in range (1,4):
    driver.get(str(a+str(page)))
    time.sleep(3)
    url_tags = driver.find_elements(By.XPATH,"//h2[@class = 'a-size-mini a-spacing-none a-color-base s-line-clamp-4' or @class = 'a-size-mini a-spacing-none a-color-base s-line-clamp-2']/a")
    for ur in url_tags:
        url.append(ur.get_attribute('href'))
len(url)

153

In [8]:
#Extracting data for each product
brand_name = []
product_name = []
price_list = []
return_list = []
expected_delivery = []
availability = []
product_url = []

for product in url:
    driver.get(product)
    time.sleep(3)
    
    #Extracting the brand name
    
    try:
        brand_tags = driver.find_element(By.XPATH,"//h1[@class = 'a-size-large a-spacing-none']/span")
        brand_name.append(brand_tags.text.split()[:1])
    except NoSuchElementException:
        brand_name.append('-')
    
    #Extracting the product name
    try:
        product_tags = driver.find_element(By.XPATH,"//h1[@class = 'a-size-large a-spacing-none']/span")
        product_name.append(product_tags.text)
    except NoSuchElementException:
        product_name.append('-')
            
    #Extracting the price
    try:
        price_tags = driver.find_element(By.XPATH,"//span[@id='priceblock_ourprice']")
        price_list.append(price_tags.text)
    except NoSuchElementException:
        price_list.append('-')
            
    #Extracting return/exchange information
    try:
        return_tags = driver.find_element(By.XPATH,"//div[@data-cel-widget = 'RETURNS_POLICY']/span/div[2]/a")
        return_list.append(return_tags.text)
    except NoSuchElementException:
        return_list.append('-')
            
    #Extracting expected delivery date
    try:
        expected_tags = driver.find_element(By.XPATH,"//div[@id='ddmDeliveryMessage']/b")
        expected_delivery.append(expected_tags.text)
    except NoSuchElementException:
        expected_delivery.append('-')

    #Extracting Availability
    try:
        availability_tags = driver.find_element(By.XPATH,"//div[@id='availability']/span")
        availability.append(availability_tags.text)
    except NoSuchElementException:
        availability.append('-')

print(len(brand_name))
print(len(product_name))
print(len(price_list))
print(len(return_list))
print(len(expected_delivery))
print(len(availability))
print(len(url))

153
153
153
153
153
153
153


In [9]:
amazon_products = pd.DataFrame({})

amazon_products['Brand'] = brand_name
amazon_products['Product Name'] = product_name
amazon_products['Price'] = price_list
amazon_products['Retur/Exchange'] = return_list
amazon_products['Expected Delivery'] = expected_delivery
amazon_products['Availability'] = availability
amazon_products['Product URL'] = url

amazon_products.head(10)


,Brand,Product Name,Price,Retur/Exchange,Expected Delivery,Availability,Product URL
0,[Juârez],"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",-,7 days Replacement,-,In stock,https://www.amazon.in/Juarez-Acoustic-Cutaway-...
1,[Intern],"Intern INT-38C Right hand Acoustic Guitar Kit,...",-,7 days Replacement,-,In stock,https://www.amazon.in/Intern-INT-38C-Acoustic-...
2,[Yamaha],"Yamaha F280 Acoustic Rosewood Guitar (Natural,...",-,7 days Replacement,-,In stock,https://www.amazon.in/Yamaha-F280-Acoustic-Gui...
3,[Juarez],"Juarez Lindenwood Acoustic Guitar Kit, 38 Inch...",-,7 days Replacement,-,In stock,https://www.amazon.in/JUAREZ-JRZ38C-Acoustic-S...
4,[Juarez],"Juarez Lindenwood Acoustic Guitar Kit, 38 Inch...",-,7 days Replacement,-,In stock,https://www.amazon.in/JUAREZ-JRZ38C-Acoustic-S...
5,[Henrix],Henrix 38C 38 Inch Cutaway Acoustic Guitar Wit...,-,7 days Replacement,-,In stock,https://www.amazon.in/Henrix-Acoustic-Guitar-D...
6,[Intern],Intern INT-38C Acoustic Guitar Kit (Blue),-,7 days Replacement,-,In stock,https://www.amazon.in/Intern-INT-38C-Acoustic-...
7,[Intern],Intern 38C Wooden Acoustic Steel-String Guitar...,-,7 days Replacement,-,In stock,https://www.amazon.in/Intern-Acoustic-Steel-st...
8,[Revel],Revel 38 Inches Cutaway Design Acoustic Guitar...,-,7 days Replacement,-,In stock,https://www.amazon.in/Revel-RVL-38C-LGP-3TS-Li...
9,[Neowood],"Neowood FLYM, Acoustic Guitar with Touch Pikup...",-,7 days Returnable,-,In stock,https://www.amazon.in/Neowood-FLYM-Acoustic-Pl...


In [10]:
#Saving the DataFrame as csv.

amazon_products.to_csv('Amazon Products.csv')

In [11]:
driver.close()

## 3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [12]:
#import necessary libraries
from bs4 import BeautifulSoup
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

import warnings
warnings.filterwarnings('ignore')

In [13]:
#connect to chrome
driver = webdriver.Chrome('chromedriver.exe')

In [14]:
#User defined function to scrape 10 images from images.google.com for the search keyword
 
def search_google(keyword):  
    driver.get('http://www.images.google.com')
    
    #Accessing the search bar
    input_button = driver.find_element(By.XPATH,'//textarea[@class="gLFyf"]')
    input_button.send_keys(keyword)
    
    #Clicking the search button

    search_button = driver.find_element(By.XPATH,'//span[@class="z1asCe MZy1Rb"]')
    search_button.click()
    
    #Scraping the first 10 image results
    image_tags = driver.find_elements(By.XPATH,'//div[@class=" bRMDJf islir"]/img')
    images = []
    for image in image_tags:
        images.append(image.get_attribute('src'))
    lt = images[:10]
    
    google_images = pd.DataFrame({keyword:lt})    
    return google_images

In [16]:
search_google('fruits')

,fruits


In [17]:
search_google('Machine Learning')

,Machine Learning


In [18]:
search_google('cars')

,cars


In [19]:
search_google('guitar')

,guitar


In [20]:
search_google('cake')

,cake


In [21]:
driver.close()

## 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be  scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,  “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the  details is missing then replace it by “- “. Save your results in a dataframe and CSV.


In [22]:
#Importing the libraries


import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import NoSuchElementException
import numpy as np

from selenium.webdriver.chrome.options import Options     #importing Options
chrome_options = Options()
chrome_options.add_argument('--incognito')

import warnings
warnings.filterwarnings('ignore')

In [23]:
driver = webdriver.Chrome(r"D:\CHROME DOWNLODE\chromedriver\chromedriver.exe")

In [24]:
driver.get('http://www.flipkart.com/')

#Closing the login pop-up

try:
    
    login_popup = driver.find_element(By.XPATH,"//button[@class = '_2KpZ6l _2doB4z']")
    login_popup.click()

except NoSuchElementException as ne:
    print("Exception Raised", '(NoSuchElement)')

In [25]:
#User defined function to scrape details for smartphone from www.flipkart.com

def smartphone(phone):
    driver.get('http://www.flipkart.com/')

    #Closing the login pop-up

    try:
        login_popup = driver.find_element(By.XPATH,"//button[@class = '_2KpZ6l _2doB4z']")
        login_popup.click()
    except NoSuchElementException as ne:
        print("Exception Raised", '(No popup to close)')
        
        
    #Inputting the keyword   
    input_keyword = driver.find_element(By.XPATH,"//input[@class = '_3704LK']")
    input_keyword.send_keys(phone+" smartphone")

    #Searching

    search_button = driver.find_element(By.XPATH,"//button[@class = 'L0Z3Pu']")
    search_button.click()
    
    #empty list
    brand_name = []
    smartphone_name = []
    colour_list = []
    RAM = []
    storage = []
    primary_camera = []
    secondary_camera = []
    display_size = []
    battery = []
    price  = []
    product_url = []

    time.sleep(2)
    brand_tags = driver.find_elements(By.XPATH,"//div[@class ='_4rR01T']")

    
    for brand in brand_tags:
        brand_name.append(brand.text.split()[0])
    time.sleep(1)
    print(len(brand_name))
    
    #Smart phone
    smartphone_tags = driver.find_elements(By.XPATH,"//div[@class ='_4rR01T']")

    for smart in smartphone_tags:
        smartphone_name.append(smart.text)
    time.sleep(1)
    print(len(smartphone_name))
    
    #colour
    colour_tags = driver.find_elements(By.XPATH,"//div[@class ='_4rR01T']")

    for colour in colour_tags:
        colour_list.append(colour.text.split()[-4])
    time.sleep(1)
    print(len(colour_list))

    #Ram
    ram_tags = driver.find_elements(By.XPATH,"//div[@class ='fMghEO']/ul/li[1]")

    for ram in ram_tags:
        RAM.append(ram.text[:9])
    time.sleep(1)
    print(len(RAM))
    
    #Storage
    storage_tags = driver.find_elements(By.XPATH,"//div[@class ='fMghEO']/ul/li[1]")

    for store in storage_tags:
        storage.append(store.text[-10:])
    time.sleep(1)
    print(len(storage))
    
    #Primary Camera
    primary_tags = driver.find_elements(By.XPATH,"//div[@class ='fMghEO']/ul/li[3]")
    for primary in primary_tags:
        primary_camera.append(primary.text[:17])
    time.sleep(1)
    print(len(primary_camera))
    
    #secondary Camera
    secondary_tags = driver.find_elements(By.XPATH,"//div[@class ='fMghEO']/ul/li[3]")

    for secondary in secondary_tags:
        secondary_camera.append(secondary.text[19:])
    for i in range (0,len(secondary_camera)):
        if secondary_camera[i] == '':
            secondary_camera[i] = '-'
    time.sleep(1)
    print(len(secondary_camera))
    
    #Display 
    display_tags = driver.find_elements(By.XPATH,"//div[@class ='fMghEO']/ul/li[2]")

    for display in display_tags:
        display_size.append(display.text)
    time.sleep(1)
    print(len(display_size))
    
    #Battery
    battery_tags = driver.find_elements(By.XPATH,"//div[@class ='fMghEO']/ul/li[4]")
    for bat in battery_tags:
        battery.append(bat.text)
    time.sleep(1)
    print(len(battery))
    
    #Price
    price_tags = driver.find_elements(By.XPATH, "//div[@class='_30jeq3 _1_WHN1']")
    for pr in price_tags:
        if pr.text:
            price.append(pr.text)
        else:
            price == 'Price: Not Available'
            price.append("-")
        time.sleep(3)

    print(len(price))


    url_tags = driver.find_elements(By.XPATH,"//a[@class = '_1fQZEK']")

    for url in url_tags:
        product_url.append(url.get_attribute('href'))
    time.sleep(1)
    print(len(product_url))
    
    
    #Creating DataFrame
    smartphone_data = pd.DataFrame({})
    smartphone_data['Brand '] = brand_name
    smartphone_data['Smartphone'] = smartphone_name
    smartphone_data['Colour'] = colour_list
    smartphone_data['RAM'] = RAM 
    smartphone_data['ROM(Storage)'] = storage
    smartphone_data['Primary Camera'] = primary_camera 
    smartphone_data['Secondary Camera'] = secondary_camera
    smartphone_data['Display Size'] = display_size 
    smartphone_data['Battery Capacity'] = battery
    smartphone_data['Price'] = price
    smartphone_data['Product URL'] = product_url
    smartphone_data.replace(r'^\s*$', np.nan, regex=True)
    
    return smartphone_data
    

In [28]:
pixel = smartphone('one plus')

Exception Raised (No popup to close)
24
24
24
24
24
24
24
24
24
24
24


In [29]:
pixel

,Brand,Smartphone,Colour,RAM,ROM(Storage),Primary Camera,Secondary Camera,Display Size,Battery Capacity,Price,Product URL
0,OnePlus,"OnePlus Nord CE 2 Lite 5G (Black Dusk, 128 GB)",(Black,6 GB RAM,128 GB ROM,64MP Rear Camera,-,16.74 cm (6.59 inch) Display,5000 mAh Battery,"₹18,859",https://www.flipkart.com/oneplus-nord-ce-2-lit...
1,OnePlus,"OnePlus Nord CE 2 Lite 5G (Blue Tide, 128 GB)",(Blue,6 GB RAM,128 GB ROM,64MP Rear Camera,-,16.74 cm (6.59 inch) Display,5000 mAh Battery,"₹19,296",https://www.flipkart.com/oneplus-nord-ce-2-lit...
2,OnePlus,"OnePlus 11R 5G (Galactic Silver, 128 GB)",(Galactic,8 GB RAM,128 GB ROM,50MP Rear Camera,-,17.02 cm (6.7 inch) Display,5000 mAh Battery,"₹39,849",https://www.flipkart.com/oneplus-11r-5g-galact...
3,OnePlus,"OnePlus Nord 2T 5G (Gray Shadow, 128 GB)",(Gray,8 GB RAM,128 GB ROM,50MP Rear Camera,-,17.02 cm (6.7 inch) Display,4500 mAh Battery,"₹28,695",https://www.flipkart.com/oneplus-nord-2t-5g-gr...
4,OnePlus,"OnePlus 11R 5G (Galactic Silver, 256 GB)",(Galactic,16 GB RAM,256 GB ROM,50MP Rear Camera,-,17.02 cm (6.7 inch) Display,5000 mAh Battery,"₹44,660",https://www.flipkart.com/oneplus-11r-5g-galact...
5,OnePlus,"OnePlus 11R 5G (Sonic Black, 256 GB)",(Sonic,16 GB RAM,256 GB ROM,50MP Rear Camera,-,17.02 cm (6.7 inch) Display,5000 mAh Battery,"₹44,716",https://www.flipkart.com/oneplus-11r-5g-sonic-...
6,OnePlus,"OnePlus Nord 2T 5G (Jade Fog, 128 GB)",(Jade,8 GB RAM,128 GB ROM,50MP Rear Camera,-,17.02 cm (6.7 inch) Display,4500 mAh Battery,"₹28,643",https://www.flipkart.com/oneplus-nord-2t-5g-ja...
7,OnePlus,"OnePlus 10R (Forest Green, 256 GB)",(Forest,12 GB RAM,256 GB ROM,50MP Rear Camera,-,17.02 cm (6.7 inch) Display,5000 mAh Battery,"₹40,999",https://www.flipkart.com/oneplus-10r-forest-gr...
8,OnePlus,"OnePlus 10R (Forest Green, 128 GB)",(Forest,8 GB RAM,128 GB ROM,50MP Rear Camera,-,17.02 cm (6.7 inch) Display,5000 mAh Battery,"₹32,500",https://www.flipkart.com/oneplus-10r-forest-gr...
9,OnePlus,"OnePlus 11R 5G (Sonic Black, 128 GB)",(Sonic,8 GB RAM,128 GB ROM,50MP Rear Camera,-,17.02 cm (6.7 inch) Display,5000 mAh Battery,"₹39,590",https://www.flipkart.com/oneplus-11r-5g-sonic-...


In [30]:
pixel.to_csv('Flipkart-oneplus.csv')

## 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps

In [31]:
#Importing Necessary Libraries
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import NoSuchElementException
import re

from selenium.webdriver.chrome.options import Options     #importing Options
chrome_options = Options()
chrome_options.add_argument('--incognito')

import warnings
warnings.filterwarnings('ignore')

In [32]:
#connect to chrome
driver = webdriver.Chrome(r"D:\CHROME DOWNLODE\chromedriver\chromedriver.exe")

In [33]:
driver.get('https://www.google.com/maps')

In [34]:
#User defined function to scrape the geospatial cordinates of a searched city.
def geo_coordinates(city):
    global c
    c = city
    driver.get('https://www.google.com/maps')
    #Inputting the city name
    search_input = driver.find_element(By.XPATH,"//input[@autofocus = 'autofocus']")
    search_input.send_keys(city)

    #Searching

    search_button = driver.find_element(By.XPATH,"//div[@class = 'pzfvzf']")
    search_button.click()
    time.sleep(3)
    return (driver)

In [35]:
def geo_code(geo):
    gurl = driver.current_url
    keyword = re.compile(r'https://www\.google\.com/maps/place/[a-zA-Z0-9\+,]+/@([0-9\.]+,[0-9\.-]+)')
    aurl = keyword.finditer(str(gurl))
    for i in aurl:
        stra = str(i.group(1))
        print("Geospacial Coordinates of", c," is",stra)

In [36]:
geo = geo_coordinates('dehradun')
geo_code(geo)

Geospacial Coordinates of dehradun  is 30.3256215,77.8523181


In [37]:
driver.close()

## 6. Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21)  from trak.in

In [38]:
#Importing Necessary Libraries
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.chrome.options import Options     #importing Options
chrome_options = Options()
chrome_options.add_argument('--incognito')

import warnings
warnings.filterwarnings('ignore')

In [39]:
driver = webdriver. Chrome (r"D:\CHROME DOWNLODE\chromedriver\chromedriver.exe")
#Let's maximize the automated chrome window
driver.maximize_window()
url="https://trak.in/india-startup-funding-investment-2015/"
driver.get(url)

In [40]:
Dates=[]
Company=[]
Industry=[]
Investor_Name=[]
Investment_Type=[]
Amount=[]

#scraping the company_name 
companies=driver.find_elements(By.XPATH,"//td[@class='column-3']")
for i in companies:
    if i.text is None :
        Company.append("--") 
    else:
        Company.append(i.text)
print(len(Company))

#scraping the Industry 
Ind=driver.find_elements(By.XPATH,"//td[@class='column-4']")
for i in Ind:
    if i.text is None :
        Industry.append("--") 
    else:
        Industry.append(i.text)
print(len(Industry))

#scraping the Dates 
dt=driver.find_elements(By.XPATH,"//td[@class='column-2']")
for i in dt:
    if i.text is None :
        Dates.append("--") 
    else:
        Dates.append(i.text)
print(len(Dates))

#scraping the Investor_Name 
IN=driver.find_elements(By.XPATH,"//td[@class='column-7']")
for i in IN:
    if i.text is None :
        Investor_Name.append("--") 
    else:
        Investor_Name.append(i.text)
print(len(Investor_Name))

#scraping the Investment_Type 
IT=driver.find_elements(By.XPATH,"//td[@class='column-8']")
for i in IT:
    if i.text is None :
        Investment_Type.append("--") 
    else:
        Investment_Type.append(i.text)
print(len(Investment_Type))

#scraping the Amount 
Price=driver.find_elements(By.XPATH,"//td[@class='column-9']")
for i in Price:
    if i.text is None :
        Amount.append("--") 
    else:
        Amount.append(i.text)
print(len(Amount))



392
392
392
392
392
392


In [41]:
Funding=pd.DataFrame()
Funding['Company']=Company
Funding['Industry']=Industry
Funding['Investor_Name']=Investor_Name
Funding['Amount Invested']=Amount
Funding['Specification']=Investment_Type
Funding['Dates']=Dates

In [42]:
Funding

,Company,Industry,Investor_Name,Amount Invested,Specification,Dates
0,BYJU’S,Edu-tech,Innoven Capital,"460,000,000",Series F,01/04/2021
1,Meesho,E-commerce,SoftBank Vision Fund 2,"300,000,000",Series E,05/04/2021
2,Swiggy,Online Food Delivery,"Amansa Holdings, Carmignac, Falcon Edge Capita...","343,000,000",Series J,14/04/2021
3,Groww,FinTech,"MC Global Edtech, B Capital, Baron, others","83,000,000",Series D,07/04/2021
4,Beldara,E-commerce,Hindustan Media Ventures,"7,400,000",Venture,14/04/2021
...,...,...,...,...,...,...
387,zippserv,Consumer Internet,Info Edge (India) Ltd,"440,000",Seed/ Angel Funding,09/07/2018
388,Groww,Consumer Internet,"Insignia Ventures Partners, Lightbridge Partne...","1,100,000",Seed/ Angel Funding,09/07/2018
389,Avenue Growth,Consumer Internet,"Avtar Monga, chief operating office at IDFC Ba...","437,000",Seed/ Angel Funding,10/07/2018
390,iNICU,Healthcare,Venture Catalysts,N/A,Seed/ Angel Funding,10/07/2018


In [43]:
driver.close()

## 7. Write a program to scrap all the available details of best gaming laptops from digit.in.


In [44]:
#Importing the libraries

from bs4 import BeautifulSoup
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.chrome.options import Options     #importing Options
chrome_options = Options()
chrome_options.add_argument('--incognito')

import warnings
warnings.filterwarnings('ignore')
from selenium.webdriver.common.keys import Keys

In [45]:
driver = webdriver.Chrome('chromedriver.exe')

In [46]:
driver.get("https://www.digit.in/")

In [47]:
search = driver.find_element(By.XPATH,'/html/body/div[2]/div/ul/li[7]/span')
search.click()



In [48]:
laptop = driver.find_element(By.XPATH,'/html/body/div[2]/div/ul/li[7]/div/div/div[2]/div/ul[1]/li[2]/a')
laptop.click()

In [49]:
best_gaming_laptop = driver.find_element(By.XPATH,'/html/body/div[7]/div/div/div[2]/div[1]/ul/li[9]/a')
best_gaming_laptop.click()

In [50]:
#Extracting the urls for products from first page.
url = []

url_tags = driver.find_elements(By.XPATH,'//span[@class="datahreflink"]')
for ur in url_tags:
    url.append(ur.get_attribute('data-href'))
print(len(url))

37


In [51]:
##Extracting the details 
brand_name = []
product_name = []
processor_name = []
os_name = []
display = []
memory = []
price = []
release = []
details = []

for i in url:
    driver.get(i)
    time.sleep(3)
    
    #Getting the brand name
    try:
        brand_name.append(driver.find_element(By.XPATH,"//div[@class = 'status_bar']/div[3]/strong").text)
    except NoSuchElementException:
        brand_name.append("-")
  
    #Getting the product name
    try:
        product_name.append(driver.find_element(By.XPATH,"//div[@class = 'heading-wraper']/h1").text)
    except NoSuchElementException:
        product_name.append("-")
   
    #Getting the processor details
    try:
        processor_name.append(driver.find_element(By.XPATH,"//div[@class = 'specs_details ']/ul/li[3]/div/p/strong").text)
    except NoSuchElementException:
        processor_name.append("-")
        
    #Getting the OS details
    try:
        os_name.append(driver.find_element(By.XPATH,"//div[@class = 'specs_details ']/ul/li[1]/div/p/strong").text)
    except NoSuchElementException:
        os_name.append("-")
  
    #Getting the display details
    try:
        display.append(driver.find_element(By.XPATH,"//div[@class = 'specs_details ']/ul/li[2]/div/p/strong").text)
    except NoSuchElementException:
        display.append("-")
    
    #Getting the memory details
    try:
        memory.append(driver.find_element(By.XPATH,"//div[@class = 'specs_details ']/ul/li[4]/div/p/strong").text)
    except NoSuchElementException:
        memory.append("-")

    #Getting the price details
    try:
        price.append(driver.find_element(By.XPATH,"//div[@class = 'price_sec']/div/h2/strong").text)
    except NoSuchElementException:
        price.append("-")

    #Getting the release date
    try:
        release.append(driver.find_element(By.XPATH,"//div[@class = 'status_bar']/div[2]/strong").text)
    except NoSuchElementException:
        release.append("-")

    #Getting the Other details
    try:
        details.append(driver.find_element(By.XPATH,"//div[@class = 'common_container']/div[2]/p").text)
    except NoSuchElementException:
        details.append("-")
        
print(len(brand_name))        
print(len(product_name))
print(len(processor_name))
print(len(os_name))
print(len(display))
print(len(memory))
print(len(price))
print(len(release))
print(len(details))
    
gaming_laptops = pd.DataFrame({})
gaming_laptops['Brand'] = brand_name
gaming_laptops['Laptop Name'] = product_name
gaming_laptops['Processor'] = processor_name
gaming_laptops['OS'] = os_name
gaming_laptops['Display Size'] = display
gaming_laptops['Memory'] = memory
gaming_laptops['Price'] = price
gaming_laptops['Release Date'] = release
gaming_laptops['Availability in India'] = details

    
gaming_laptops

37
37
37
37
37
37
37
37
37


,Brand,Laptop Name,Processor,OS,Display Size,Memory,Price,Release Date,Availability in India
0,HP,HP Omen 17-ck2008TX 13th Gen Core i7-13700HX,13th Gen Intel Core i7-13700HX | 5.0 GHz,Windows 11 Home,"17.3"" (2560 x 1440)",1 TB SSD/16 GB DDR5,"₹ 269,777","12 May, 2023",-
1,HP,HP Omen 17-ck2008TX 13th Gen Core i7-13700HX,13th Gen Intel Core i7-13700HX | 5.0 GHz,Windows 11 Home,"17.3"" (2560 x 1440)",1 TB SSD/16 GB DDR5,"₹ 269,777","12 May, 2023",-
2,-,-,-,-,-,-,-,-,-
3,HP,HP Omen 17-ck2008TX 13th Gen Core i7-13700HX,13th Gen Intel Core i7-13700HX | 5.0 GHz,Windows 11 Home,"17.3"" (2560 x 1440)",1 TB SSD/16 GB DDR5,"₹ 269,777","12 May, 2023",-
4,-,-,-,-,-,-,-,-,-
5,MSI,MSI GT77 Titan 12UHS-054IN 12th Gen Core i9-12...,12th Gen Intel Core i9-12900HX | 3.6 GHz,Windows 11 Home,"17.3"" (3840 x 2160)",2 TB SSD/64 GB DDR5,"₹ 499,990","31 Oct, 2022",MSI GT77 Titan 12UHS-054IN 12th Gen Core i9-12...
6,MSI,MSI GT77 Titan 12UHS-054IN 12th Gen Core i9-12...,12th Gen Intel Core i9-12900HX | 3.6 GHz,Windows 11 Home,"17.3"" (3840 x 2160)",2 TB SSD/64 GB DDR5,"₹ 499,990","31 Oct, 2022",MSI GT77 Titan 12UHS-054IN 12th Gen Core i9-12...
7,-,-,-,-,-,-,-,-,-
8,MSI,MSI GT77 Titan 12UHS-054IN 12th Gen Core i9-12...,12th Gen Intel Core i9-12900HX | 3.6 GHz,Windows 11 Home,"17.3"" (3840 x 2160)",2 TB SSD/64 GB DDR5,"₹ 499,990","31 Oct, 2022",MSI GT77 Titan 12UHS-054IN 12th Gen Core i9-12...
9,-,-,-,-,-,-,-,-,-


In [52]:
driver.close()

## 8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be  scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.


In [53]:
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.chrome.options import Options     #importing Options
chrome_options = Options()
chrome_options.add_argument('--incognito')

import warnings
warnings.filterwarnings('ignore')
from selenium.webdriver.common.keys import Keys

In [61]:
driver = webdriver.Chrome('chromedriver.exe')

In [62]:
driver.get('https://www.forbes.com/billionaires/')

In [64]:
Name =[]
Age =[]
Source =[]
Industry =[]
Citizenship=[]
Net_worth=[]
Rank=[]
name_tag=driver.find_elements(By.XPATH,'//div[@class="Table_dataCell__2QCve"]')
for x in name_tag:
    Name.append(x.text)
    
Name= Name[0:801:4]
age_tag=driver.find_elements(By.XPATH,'//div[@class="Table_dataCell__2QCve"]')
for x in name_tag:
    Age.append(x.text)
Age = Age[1:801:4]
import time
Sources=[]
# scraping the data of source of income of the billionaires
sour_tag = driver.find_elements(By.CLASS_NAME,"Table_dataCell__2QCve")
for sour in sour_tag:
    Sources.append(sour.text)
time.sleep(1)

Industrys=[]
# scraping data of industry of the billionaires
ind_tag = driver.find_elements(By.CLASS_NAME,"Table_dataCell__2QCve")
for ind in ind_tag:
    Industrys.append(ind.text)
time.sleep(1)
industry = (Industrys[4:1201:6])
sources = Sources[5:1201:6]
citizenship = driver.find_elements(By.XPATH,'//div[@class="TableRow_cell__db-hv Table_cell__houv9"]')
for i in citizenship:
    Citizenship.append(i.text)
Citizenship= Citizenship[4:1401:7]

networth = driver.find_elements(By.XPATH,'//div[@class="TableRow_cell__db-hv Table_cell__houv9"]')
for i in networth:
    Net_worth.append(i.text)
Net_worth= Net_worth[2:1401:7]
rank = driver.find_elements(By.XPATH,'//div[@class="TableRow_cell__db-hv Table_cell__houv9"]')
for i in rank:
    Rank.append(i.text)
Rank= Rank[0:1401:7]


# Define the data
data = {
    "Rank":Rank,
    "Name": Name,
    "Net worth": Net_worth,
    "Age": Age,
    "Citizenship": Citizenship,
    "Source": sources,
    "Industry": industry
}

df = pd.DataFrame(data)
df

,Rank,Name,Net worth,Age,Citizenship,Source,Industry
0,1,Bernard Arnault & family,$211 B,74,France,Fashion & Retail,LVMH
1,2,Elon Musk,$180 B,51,United States,Automotive,"Tesla, SpaceX"
2,3,Jeff Bezos,$114 B,59,United States,Technology,Amazon
3,4,Larry Ellison,$107 B,78,United States,Technology,Oracle
4,5,Warren Buffett,$106 B,92,United States,Finance & Investments,Berkshire Hathaway
...,...,...,...,...,...,...,...
195,195,Jin Baofang,$9.6 B,70,China,Energy,Solar panels
196,195,Luo Liguo & family,$9.6 B,67,China,Manufacturing,Chemicals
197,195,Marijke Mars,$9.6 B,58,United States,Food & Beverage,"Candy, pet food"
198,195,Pamela Mars,$9.6 B,62,United States,Food & Beverage,"Candy, pet food"


In [65]:
driver.close()

## 9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted  from any YouTube Video.

In [66]:
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.chrome.options import Options     #importing Options
chrome_options = Options()
chrome_options.add_argument('--incognito')

import warnings
warnings.filterwarnings('ignore')
from selenium.webdriver.common.keys import Keys

In [67]:
driver = webdriver.Chrome('chromedriver.exe')

In [68]:
driver.get('https://www.youtube.com/')

In [72]:
# entering the designation as required in the question
designation = driver.find_element(By.XPATH,'//input[@class="ytd-searchbox"]')
designation.send_keys("Avatar 2")

In [73]:
# clicking on search button to search the required data 
search = driver.find_element(By.XPATH,'/html/body/ytd-app/div[1]/div/ytd-masthead/div[4]/div[2]/ytd-searchbox/button/yt-icon')
search.click()

In [74]:
# clicking on search button to search the required data 
first_video  = driver.find_element(By.XPATH,'/html/body/ytd-app/div[1]/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-video-renderer[1]/div[1]/div/div[1]/div/h3/a/yt-formatted-string')
first_video .click()

In [78]:
#Extracting the first 100 comments, upvote and time when comment was posted.
for i in range(1,5):

    comments_list = []

    comment_tags = driver.find_elements(By.XPATH,'//yt-formatted-string[@class="style-scope ytd-comment-renderer"]')
    for com in comment_tags[:500]:
        comments_list.append(com.text)
    

    upvotes_list = []
    upvotes_tags = driver.find_elements(By.ID,"vote-count-middle")
    for upvote in upvotes_tags[:500]:
        upvotes_list.append(upvote.text)
    

    time_posted = []
    time_tags = driver.find_elements(By.XPATH,'//yt-formatted-string[@class="published-time-text style-scope ytd-comment-renderer"]')
    for time in time_tags[:500]:
        time_posted.append(time.text)
    
    while(True):
        height = driver.execute_script("return document.body.scrollHeight")
        driver.find_element(By.TAG_NAME,'body').send_keys(Keys.END)
        if int(height)==0:
            break


    
    
print(len(comments_list))
print(len(upvotes_list))
print(len(time_posted))

500
500
500


In [79]:
#Creating the dataframe
youtube_comments = pd.DataFrame({})
youtube_comments['Comment'] = comments_list
youtube_comments['Upvotes'] = upvotes_list
youtube_comments['Time Posted'] = time_posted

youtube_comments


,Comment,Upvotes,Time Posted
0,You know the film is really good when you come...,10K,4 months ago
1,"This film was magnificent. No cheesy jokes, ju...",1.2K,3 months ago
2,"it was worth every minute, i didn’t expect to ...",1.8K,4 months ago
3,I have never once went to a theatre more than ...,463,3 months ago
4,Never realized how stunning this trailer was. ...,341,3 months ago
...,...,...,...
495,I loved how Jake turned out to a alian and the...,4,4 months ago
496,i watched the film in 3D i was living a dream,1,3 months ago
497,"God, the water animation is absolutely gloriou...",560,6 months ago
498,I'm going to watch this in theaters tomorrow o...,4,3 months ago


In [80]:
driver.close()

## 10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in  “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews,  overall reviews, privates from price, dorms from price, facilities and property description

In [81]:
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.chrome.options import Options     #importing Options
chrome_options = Options()
chrome_options.add_argument('--incognito')

import warnings
warnings.filterwarnings('ignore')
from selenium.webdriver.common.keys import Keys

In [103]:
driver = webdriver.Chrome('chromedriver.exe')

In [104]:
driver.get('https://www.hostelworld.com/')

In [110]:
# entering the designation as required in the question
designation = driver.find_element(By.XPATH,'//input[@class="search-input"]')
designation.send_keys("London")

In [111]:
# clicking on search button to search the required data 
search = driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[2]/div[2]/div/div/div[4]/div/div[2]/div/div[1]/div/div/ul/li[2]/div')
search.click()

In [112]:
# clicking on search button to search the required data 
search = driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[2]/div[2]/div/div/div[4]/div/div[2]/div/div[5]/button')
search.click()

In [137]:
#extracting data
hotel_name=[]
name=driver.find_elements(By.XPATH,'//div[@class="property-name"]')
for x in name[:20]:
    hotel_name.append(x.text)
print(len(hotel_name))

distance=[]
dis=driver.find_elements(By.XPATH,'//span[@class="distance-description"]')
for x in dis[:20]:
    distance.append(x.text)
print(len(distance))

ratings=[]
rate=driver.find_elements(By.XPATH,'//div[@class="property-rating"]')
for x in rate[:20]:
    ratings.append(x.text.split('\n')[0])
print(len(ratings))

reviews=[]
review=driver.find_elements(By.XPATH,'//span[@class="left-margin"]')
for x in review[:20]:
    reviews.append(x.text)
print(len(reviews))

overall_review=[]
over=driver.find_elements(By.XPATH,'//div[@class="keyword"]')
for x in over[:20]:
    overall_review.append(x.text)
print(len(overall_review))

privets=[]
privet=driver.find_elements(By.XPATH,'//div[@class="accommodation-price unavailable"]')
for x in privet[:20]:
    privets.append(x.text)
print(len(privets))

dorms_form=[]
dorms=driver.find_elements(By.XPATH,'//strong[@class="current"]')
for x in dorms[:20]:
    dorms_form.append(x.text)
print(len(dorms_form))

df=pd.DataFrame({
    
    'Hostel Name':hotel_name,
    'Distance from City Centre':distance,
    'Ratings':ratings,
    'Total Reviews':reviews,
    'Overall Review':overall_review,
    'Privates from Price':privets,
    'Dorms from Price':dorms_form
})


20
20
20
20
20
20
20


In [138]:
df

,Hostel Name,Distance from City Centre,Ratings,Total Reviews,Overall Review,Privates from Price,Dorms from Price
0,St Christopher's Village,- 3.6km from city centre,7.7,(14789),Very Good,No Privates Available,"₹6,070"
1,Wombat's City Hostel London,- 5.5km from city centre,9.0,(1996),Superb,No Privates Available,"₹18,395"
2,Hostel One Notting Hill,- 1.8km from city centre,9.6,(7455),Superb,No Privates Available,"₹6,259"
3,St Christopher's Village,- 3km from city centre,7.7,(740),Very Good,No Privates Available,"₹4,152"
4,Generator London,- 5.4km from city centre,7.6,(4919),Very Good,No Privates Available,"₹5,145"
5,Urbany Hostel London,- 1.7km from city centre,9.3,(1504),Superb,No Privates Available,"₹6,722"
6,Safestay London Elephant & Castle,- 5.8km from city centre,7.6,(3991),Very Good,No Privates Available,"₹3,487"
7,Safestay London Kensington Holland Park,- 9km from city centre,6.9,(4454),Good,No Privates Available,"₹2,161"
8,No.8 Seven Sisters,- 11.9km from city centre,6.1,(4228),Good,No Privates Available,"₹1,139"
9,London Backpackers,- 7.5km from city centre,7.8,(388),Very Good,No Privates Available,"₹6,223"
